In [1]:
import pytesseract
from PIL import Image
from pydantic import BaseModel, Field, ValidationError
from datetime import datetime
from langchain import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import json

In [2]:
# Certifique-se de que o pytesseract pode encontrar o executável do Tesseract (para Windows)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


In [3]:

# Modelo Pydantic para armazenar os resultados da análise do dashboard
class DashboardAnalysis(BaseModel):
    name: str = Field(..., description="O nome ou título do dashboard")
    timestamp: datetime = Field(..., description="O momento em que a imagem foi analisada")
    summary: str = Field(..., description="Resumo do conteúdo do dashboard")
    key_metrics: dict = Field(..., description="Principais métricas extraídas do dashboard")

    # Método customizado para converter o datetime
    def model_dump_custom(self):
        data = self.model_dump()
        data['timestamp'] = self.timestamp.isoformat()  # Converte datetime para string no formato ISO
        return data

# Função para realizar OCR usando pytesseract
def extract_text_from_image(image_path):
    img = Image.open(image_path)
    # Realiza OCR usando pytesseract
    text = pytesseract.image_to_string(img)
    return text

# Função para processar o texto do dashboard com o agente LangChain
def process_text_with_agent(text):
    # Inicializa o LangChain com o modelo OpenAI (certifique-se de ter configurado sua chave de API)
    llm = OpenAI()

    # Cria um template de prompt para a sumarização
    prompt_template = PromptTemplate(
        input_variables=["text"],
        template="Analise os seguintes dados do dashboard e resuma os principais insights:\n{text}\n"
    )

    # Cria uma cadeia LLM
    chain = LLMChain(llm=llm, prompt=prompt_template)

    # Usa a cadeia para analisar o texto
    response = chain.run(text)
    return response

# Função para criar um objeto Pydantic para o relatório de análise
def create_dashboard_analysis(name, summary, key_metrics):
    timestamp = datetime.now()

    try:
        analysis = DashboardAnalysis(
            name=name,
            timestamp=timestamp,
            summary=summary,
            key_metrics=key_metrics
        )
        return analysis
    except ValidationError as e:
        print("Erro de validação:", e)
        return None

# Script principal
def main(image_path):
    # Etapa 1: Extrair texto da imagem (OCR básico)
    extracted_text = extract_text_from_image(image_path)
    print("Texto extraído:", extracted_text)

    # Etapa 2: Processar o texto extraído com o agente LangChain para resumir
    summary = process_text_with_agent(extracted_text)
    print("Resumo do Agente LangChain:", summary)

    # Etapa 3: Criar objeto Pydantic com os dados extraídos
    dashboard_analysis = create_dashboard_analysis("Análise do Dashboard", summary, {})

    if dashboard_analysis:
        # Usa o dump customizado para tratar a serialização do datetime
        print("Relatório de Análise do Dashboard:", json.dumps(dashboard_analysis.model_dump_custom(), indent=4))
    else:
        print("Falha ao criar o objeto DashboardAnalysis")



In [4]:
image_path = "Screenshot_1.png"  # Forneça o caminho da imagem do dashboard
main(image_path)

Texto extraído: Receita Bruta RS(MM)

DRE Grafica (=)

4.09403
= 3.64429
449,74
898,08 +972
27621 807

Receita Dedugdes Receita Custos Result. Result.
Bruta Uiquida  e = Operac. Financ.
Despesas

Ll

Receita Bruta: Previsto vs Realizado - jan-2021 - dez-2023

Jan Abr Jul ‘Out
9 2021

94,03 r ) Al e@

4.110,79 (-0,41%) 2.950,14 (-6,91%)

—
23928

Rateio

“5522

custo
Financ.

e Despesas RS(MM)

» |

60523,

Result.
antes
IR

iItado Operacional R$(MM) do Antes do IR R$(MM)
08 @ 23 @
713,28 (25,91%) 366,78 (65,01%)
Pareto por Segmentacéo
2728 947% 974% 100,0%
823%
ets) 505
(MEGA STORE ‘ALIMENTAGAO CALGADO ROUPA LIVRARIA,
Jul Out Jan Abr Jul Out
a



C:\Users\PedroLustosa\AppData\Local\Temp\ipykernel_15824\1309857031.py:24: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI()
C:\Users\PedroLustosa\AppData\Local\Temp\ipykernel_15824\1309857031.py:33: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)
C:\Users\PedroLustosa\AppData\Local\Temp\ipykernel_15824\1309857031.py:36: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(text)


Resumo do Agente LangChain: Principais insights:

1. A receita bruta da empresa teve um crescimento de aproximadamente 898,08 milhões de reais em dezembro de 2023, comparado a janeiro de 2021. Isso representa uma variação de 22% no período analisado.

2. A receita líquida da empresa teve um crescimento de aproximadamente 972 milhões de reais em dezembro de 2023, comparado a janeiro de 2021. Isso representa uma variação de 30% no período analisado.

3. A empresa teve um resultado líquido positivo em todos os meses analisados, com um pico em outubro de 2023, atingindo uma receita líquida de 3.64429 milhões de reais.

4. As despesas com custos e financeiras foram as principais responsáveis pela diminuição do resultado líquido da empresa, representando 25,91% e 65,01% do resultado antes do IR em janeiro de 2021, respectivamente.

5. O segmento de Mega Store foi o principal responsável pela receita bruta da
Relatório de Análise do Dashboard: {
    "name": "An\u00e1lise do Dashboard",
    "t